In [ ]:
# Enable autoreload for development
%load_ext autoreload
%autoreload 2

In [ ]:
# Set project root and add src to path
import sys
from pathlib import Path
import os

PROJECT_ROOT = '/scratch/edk202/word2gm-fast'
project_root = Path(PROJECT_ROOT)
src_path = project_root / 'src'

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

In [ ]:
# Print resource summary
from word2gm_fast.utils.resource_summary import print_resource_summary

print_resource_summary()

E0000 00:00:1752860241.628617 4191902 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752860241.696337 4191902 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752860242.430524 4191902 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752860242.430549 4191902 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752860242.430551 4191902 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752860242.430552 4191902 computation_placer.cc:177] computation placer already registered. Please check linka

<pre>SYSTEM RESOURCE SUMMARY
=============================================
Hostname: gr035.hpc.nyu.edu

Job Allocation:
   CPUs: 14
   Memory: 125.0 GB
   Partition: rtx8000
   Job ID: 63887006
   Node list: gr035

Physical GPU Hardware:
   Physical GPUs available: 1
   GPU 0: Quadro RTX 8000
      Memory: 0.5/45.0 GB (44.5 GB free)
      Temperature: 27°C
      Utilization: GPU 0%, Memory 0%

TensorFlow GPU Recognition:
   TensorFlow can access 1 GPU(s)
      /physical_device:GPU:0
   Built with CUDA support: True
=============================================</pre>

In [ ]:
# Define paths for your corpus artifacts and output
dataset_artifacts_dir = (
    '/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data/'
    '1850_artifacts'
)
output_dir = '/scratch/edk202/word2gm-fast/output/test_corpus'
Path(output_dir).mkdir(parents=True, exist_ok=True)

# Set TensorBoard log directory
tensorboard_log_dir = output_dir + '/tensorboard'

In [ ]:
from word2gm_fast.io.artifacts import load_pipeline_artifacts

# Load pipeline artifacts (vocab, triplets, etc.)
artifacts = load_pipeline_artifacts(dataset_artifacts_dir)
token_to_index_table = artifacts['token_to_index_table']
index_to_token_table = artifacts['index_to_token_table']
triplets_ds = artifacts['triplets_ds']
vocab_size = artifacts['vocab_size']

<span style='font-family: monospace; font-size: 120%; font-weight: normal;'>Loading pipeline artifacts from: /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data/1850_artifacts</span>

<span style='font-family: monospace; font-size: 120%; font-weight: normal;'>Loading token-to-index vocabulary TFRecord from:<br>&nbsp;&nbsp;/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data/1850_artifacts/vocab.tfrecord</span>

I0000 00:00:1752860287.860799 4191902 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 44333 MB memory:  -> device: 0, name: Quadro RTX 8000, pci bus id: 0000:86:00.0, compute capability: 7.5


<span style='font-family: monospace; font-size: 120%; font-weight: normal;'>Token-to-index lookup table created successfully.<br>Table contains 33668 tokens. Processing time: 1.91s</span>

<span style='font-family: monospace; font-size: 120%; font-weight: normal;'>Loading index-to-token vocab TFRecord from:<br>&nbsp;&nbsp;/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data/1850_artifacts/vocab.tfrecord</span>

<span style='font-family: monospace; font-size: 120%; font-weight: normal;'>Index-to-token lookup table created successfully.<br>Table contains 33668 tokens. Processing time: 0.29s</span>

<span style='font-family: monospace; font-size: 120%; font-weight: normal;'>All artifacts loaded successfully!</span>

In [ ]:
from word2gm_fast.utils.tf_silence import import_tf_quietly

tf = import_tf_quietly()

# Build the dataset pipeline: cache -> shuffle -> batch -> prefetch
triplets_ds = triplets_ds.cache()
BATCH_SIZE = 1024
SHUFFLE_BUFFER_SIZE = BATCH_SIZE * 10
triplets_ds = triplets_ds.shuffle(SHUFFLE_BUFFER_SIZE)
triplets_ds = triplets_ds.batch(BATCH_SIZE)
triplets_ds = triplets_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir $tensorboard_log_dir --port 6006

Launching TensorBoard...

In [ ]:
from word2gm_fast.training.notebook_training import run_notebook_training

# Run General-Purpose Word2GM Training
# Hardcoded stable configuration for reliable training

print(f"Starting Word2GM training...")
print(f"Vocab size: {vocab_size}")
print(f"Output: {output_dir}")

# Run training with hardcoded stable parameters
training_results = run_notebook_training(
    training_dataset=triplets_ds,
    save_path=output_dir,
    vocab_size=vocab_size,
    embedding_size=200,           # Good balance of capacity and speed
    num_mixtures=1,               # Single Gaussian for simplicity
    spherical=True,               # Diagonal covariance
    learning_rate=0.1,            # Proven stable rate for Word2GM
    epochs=100,                    # Reasonable training duration
    adagrad=True,                 # Essential for Word2GM
    normclip=True,                # Prevents exploding gradients
    norm_cap=10.0,                # Moderate gradient clipping
    lower_sig=0.05,               # Balanced variance bounds
    upper_sig=1.5,
    wout=True,                    # Use output embeddings
    tensorboard_log_path=tensorboard_log_dir,
    monitor_interval=0.5,         # Regular monitoring
    var_scale=0.05,               # Moderate regularization
    loss_epsilon=1e-8             # Numerical stability
)

print("✅ Training completed!")

Starting Word2GM training...
Vocab size: 33668
Output: /scratch/edk202/word2gm-fast/output/test_corpus



**Word2GM Training Hyperparameters:**

| Parameter         | Value                |
|-------------------|----------------------|
| Vocab size        | `33668`  |
| Embedding size    | `200` |
| Mixtures          | `1` |
| Spherical         | `True`   |
| Learning rate     | `0.1` |
| Epochs            | `30` |
| Adagrad           | `True`     |
| Normclip          | `True`    |
| Norm cap          | `10.0`    |
| Lower sigma       | `0.05`   |
| Upper sigma       | `1.5`   |
| Wout              | `True`        |
| Var scale         | `0.05`   |
| Loss epsilon      | `1e-08`|


**Epoch 1 finished. Loss:** `0.550286`  | **Duration:** `18.38` seconds.

**Epoch 2 finished. Loss:** `0.344627`  | **Duration:** `10.15` seconds.

**Epoch 3 finished. Loss:** `0.301065`  | **Duration:** `10.09` seconds.

**Epoch 4 finished. Loss:** `0.275225`  | **Duration:** `10.08` seconds.

**Epoch 5 finished. Loss:** `0.255608`  | **Duration:** `10.11` seconds.

**Epoch 6 finished. Loss:** `0.238897`  | **Duration:** `10.12` seconds.

**Starting epoch 7/30...**